In [1]:
import pandas as pd
import requests
import numpy
import sqlite3
import os
import time

In [2]:
# Get my building blocks
Item_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Item URLs.csv")
Resource_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Resource URLs.csv")
Artefact_Journal_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Artefact and Journal URLs.csv")
Recipies = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Albion_Equipment_Recipies_3.csv")

In [3]:
# this function lets us create a dataframe out of a list of URL's for Albion Data Project GET API.
def Get_Data2(URL_list):
    df_data = pd.DataFrame(columns=['item_id', 'city', 'quality', 'sell_price_min', 'sell_price_min_date', 'sell_price_max', 'sell_price_max_date', 'buy_price_min', 'buy_price_min_date', 'buy_price_max', 'buy_price_max_date'])
    for i in range(len(URL_list)):
        print(f'working on URL #{i}')
        Current_URL = URL_list['URL'][i]
        URL_Contents = requests.get(url=Current_URL).json()
        URL_Contents = pd.DataFrame(URL_Contents)
        df_data = pd.concat([df_data,URL_Contents])
    print('API calls done')
    return df_data

# a function to take old and new prices, and drop duplicates, with the duplicates kept being the data from most recent prices.
def Update_Prices(hitorical_prices, updated_prices):
    merged = pd.concat([hitorical_prices, updated_prices], ignore_index=True)
    merged.drop_duplicates(subset=['item_id', 'city', 'quality'], keep='last', inplace=True)
    return merged

In [4]:
def Get_Data2(URL_list):
    df_data = pd.DataFrame(columns=['item_id', 'city', 'quality', 'sell_price_min', 'sell_price_min_date', 'sell_price_max', 'sell_price_max_date', 'buy_price_min', 'buy_price_min_date', 'buy_price_max', 'buy_price_max_date'])
    for i in range(len(URL_list)):
        print(f'working on URL #{i}')
        Current_URL = URL_list['URL'][i]
        URL_Contents = requests.get(url=Current_URL).json()
        URL_Contents = pd.DataFrame(URL_Contents)
        df_data = pd.concat([df_data,URL_Contents])
    print('API calls done')
    return df_data

In [5]:
# Make a fresh call to API to get recent data.
t1= time.time()
resource_prices_update = Get_Data2(Resource_URLs)
artefact_and_journal_prices_update = Get_Data2(Artefact_Journal_URLs)
t2= time.time()
diff = t2-t1
print(diff)
time.sleep(62-diff) #There is a 180 calls per minute limit that we must avoid. also 300 per 5 minutes limit.
item_prices_update = Get_Data2(Item_URLs)

working on URL #0
working on URL #1
working on URL #2
API calls done
working on URL #0
working on URL #1
working on URL #2
working on URL #3
working on URL #4
working on URL #5
working on URL #6
working on URL #7
working on URL #8
working on URL #9
working on URL #10
working on URL #11
working on URL #12
working on URL #13
working on URL #14
working on URL #15
working on URL #16
working on URL #17
working on URL #18
working on URL #19
working on URL #20
working on URL #21
working on URL #22
working on URL #23
working on URL #24
working on URL #25
working on URL #26
working on URL #27
working on URL #28
working on URL #29
working on URL #30
working on URL #31
API calls done
6.973673582077026
working on URL #0
working on URL #1
working on URL #2
working on URL #3
working on URL #4
working on URL #5
working on URL #6
working on URL #7
working on URL #8
working on URL #9
working on URL #10
working on URL #11
working on URL #12
working on URL #13
working on URL #14
working on URL #15
workin

In [6]:
#items that have no data are present but just with 0's in price data.
item_prices_update = item_prices_update[item_prices_update['sell_price_min']>0]
resource_prices_update = resource_prices_update[resource_prices_update['sell_price_min']>0]
artefact_and_journal_prices_update = artefact_and_journal_prices_update[artefact_and_journal_prices_update['sell_price_min']>0]


In [7]:
# Get prices used in last pull:
resource_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv")
artefact_and_journal_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv")
item_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv")


In [8]:
#lets update our item prices now
item_prices_final = Update_Prices(item_prices_historical, item_prices_update)
resource_prices_final = Update_Prices(resource_prices_historical, resource_prices_update)
artefact_and_journal_prices_final = Update_Prices(artefact_and_journal_prices_historical, artefact_and_journal_prices_update)


In [9]:
#and now we must save this new data into our csv files so they can be used the next time I run this program to update prices.

#top 3 are normal process, bottom 3 are to refresh the historical data files.

resource_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv", index=False)
artefact_and_journal_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv", index=False)
item_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv", index=False)

#resource_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv", index=False)
#artefact_and_journal_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv", index=False)
#item_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv", index=False)

In [10]:
# Now that we have updated our prices and saved them, we can start to manipulate the price data into our report, starting with trimming columns we dont need.

item_prices_report = item_prices_final
artefact_and_journal_prices_report = artefact_and_journal_prices_final
resource_prices_report = resource_prices_final

item_prices_report = item_prices_report[['item_id','city','sell_price_min','sell_price_min_date']]
item_prices_report = item_prices_report.groupby(['item_id', 'city']).agg({'sell_price_min_date': 'min', 'sell_price_min': 'mean'}).reset_index() #items have qualities. I looked for all qualities to "widen the net" we cast with our API call. Time to group by item so that each only has 1 row.
item_prices_report['sell_price_min'] = item_prices_report['sell_price_min'].astype(int)

artefact_and_journal_prices_report = artefact_and_journal_prices_report.groupby(['item_id']).agg({'sell_price_min_date': 'min', 'sell_price_min': 'mean'}).reset_index() #these items NEVER get checked by players, and unlike items or resources, have no attachment to geography. for this reason I check all cities and group by just item to once again "widen the net"
artefact_and_journal_prices_report['sell_price_min'] = artefact_and_journal_prices_report['sell_price_min'].astype(int)

resource_prices_report = resource_prices_report[['item_id','city','sell_price_min','sell_price_min_date']]


In [11]:
# Moving to SQL for all the merges we are about to do.
Albion_DB = "../Albion.sqlite"
Albion_conn = sqlite3.connect(Albion_DB)

item_prices_report.to_sql(name='item_prices_report', if_exists='replace', con=Albion_conn)
artefact_and_journal_prices_report.to_sql(name='artefact_and_journal_prices_report', if_exists='replace', con=Albion_conn)
resource_prices_report.to_sql(name='resource_prices_report', if_exists='replace', con=Albion_conn)
Recipies.to_sql(name='Recipies', if_exists='replace', con=Albion_conn)

4240

In [12]:
query_check_1= time.time()

Caerleon_Transport = pd.read_sql_query(
"""
WITH caerleon_prices AS (

SELECT
R.Item_Name AS "Item"
,R.Tier || '.' || R.Enchant AS Tier_Enchant


,item_id AS "item_id"
,sell_price_min AS "Caerleon Sell Price"
,sell_price_min_date "Caerleon Price Date"

FROM item_prices_report

LEFT JOIN  
    Recipies AS R ON R.Item_Placeholder = item_prices_report.item_id
    
WHERE city = 'Black Market'
)

--------------------------------------------------------------------------------
,royal_continent_prices AS (

SELECT

city AS "Buy City"
,item_id AS "item_id"
,sell_price_min AS "Buy City Price"
,sell_price_min_date "Buy City Date"

FROM item_prices_report
WHERE city IN ('Bridgewatch','Fort Sterling','Lymhurst','Martlock','Thetford')
)
--------------------------------------------------------------------------------

SELECT

caerleon_prices.*
,royal_continent_prices.*


FROM caerleon_prices

LEFT JOIN
    royal_continent_prices ON royal_continent_prices.item_id = caerleon_prices.item_id


 """, Albion_conn)

query_check_2= time.time()
print(f'{query_check_2-query_check_1} seconds to complete')

Caerleon_Transport['Profit'] = (Caerleon_Transport['Caerleon Sell Price']*0.945) - Caerleon_Transport['Buy City Price']
Caerleon_Transport['ROI'] = Caerleon_Transport['Profit']/Caerleon_Transport['Caerleon Sell Price']
Caerleon_Transport = Caerleon_Transport[['Item', 'Tier_Enchant', 'Caerleon Sell Price',
       'Caerleon Price Date', 'Buy City', 'Buy City Price',
       'Buy City Date', 'Profit', 'ROI']]
Caerleon_Transport

0.11302828788757324 seconds to complete


,Item,Tier_Enchant,Caerleon Sell Price,Caerleon Price Date,Buy City,Buy City Price,Buy City Date,Profit,ROI
0,Adept's Great Arcane Staff,4.0,7393,2023-07-21T13:20:00,Bridgewatch,6188.0,2023-07-14T12:35:00,798.385,0.107992
1,Adept's Great Arcane Staff,4.0,7393,2023-07-21T13:20:00,Fort Sterling,4867.0,2023-07-21T12:55:00,2119.385,0.286675
2,Adept's Great Arcane Staff,4.0,7393,2023-07-21T13:20:00,Lymhurst,4876.0,2023-07-20T14:00:00,2110.385,0.285457
3,Adept's Great Arcane Staff,4.0,7393,2023-07-21T13:20:00,Martlock,4454.0,2023-07-21T11:05:00,2532.385,0.342538
4,Adept's Great Arcane Staff,4.0,7393,2023-07-21T13:20:00,Thetford,6988.0,2023-07-18T09:30:00,-1.615,-0.000218
...,...,...,...,...,...,...,...,...,...
19324,Elder's Graveguard Boots,8.3,1233331,2023-05-09T05:20:00,Bridgewatch,1099998.0,2023-06-09T09:05:00,65499.795,0.053108
19325,Elder's Graveguard Boots,8.3,1233331,2023-05-09T05:20:00,Fort Sterling,1166650.0,2023-07-08T10:45:00,-1152.205,-0.000934
19326,Elder's Graveguard Boots,8.3,1233331,2023-05-09T05:20:00,Lymhurst,1178140.0,2023-07-18T13:40:00,-12642.205,-0.010250
19327,Elder's Graveguard Boots,8.3,1233331,2023-05-09T05:20:00,Martlock,1049993.0,2023-07-13T19:45:00,115504.795,0.093653


In [13]:
query_check_1= time.time()

BM_Craft = pd.read_sql_query(
"""
WITH final_query AS (
    SELECT 
    
        R.*,
        i.city AS Sell_City,
        i.sell_price_min AS Sell_Price,
        i.sell_price_min_date AS Sell_Price_Date,
        r1.city AS Main_Resource_City,
        r1.sell_price_min AS Main_Resource_Price,
        r1.sell_price_min_date AS Main_Resource_Date,
        r2.city AS Secondary_Resource_City,
        r2.sell_price_min AS Secondary_Resource_Price,
        r2.sell_price_min_date AS Secondary_Resource_Date,
        a.sell_price_min AS Artefact_Price,
        a.sell_price_min_date AS Artefact_Date,
        j1.sell_price_min AS Journal_Empty_Price,
        j2.sell_price_min AS Journal_Full_Price,
        
        CASE WHEN R.Artefact_Placeholder <> 0 AND a.sell_price_min IS NULL THEN 1 ELSE 0 END AS Missing_Artefact,
        CASE WHEN R.Secondary_Resource_Quantity <> '0' AND r2.sell_price_min IS NULL THEN 1 ELSE 0 END AS Missing_Second_Resource,
        
        R.Tier || '.' || R.Enchant AS Tier_Enchant
    FROM 
        Recipies AS R
    LEFT JOIN 
        item_prices_report AS i ON R.Item_Placeholder = i.item_id
    LEFT JOIN 
        resource_prices_report AS r1 ON R.Main_Resource = r1.item_id
    LEFT JOIN 
        resource_prices_report AS r2 ON R.Secondary_Resource = r2.item_id
    LEFT JOIN 
        artefact_and_journal_prices_report AS a ON R.Artefact_Placeholder = a.item_id
    LEFT JOIN 
        artefact_and_journal_prices_report AS j1 ON R.Journal_Empty = j1.item_id
    LEFT JOIN 
        artefact_and_journal_prices_report AS j2 ON R.Journal_Full = j2.item_id
)
SELECT 
    *,
    CASE WHEN Secondary_Resource_Price IS NULL THEN 0 ELSE Secondary_Resource_Price END AS Secondary_Resource_Price_nn,
    CASE WHEN Artefact_Price IS NULL THEN 0 ELSE Artefact_Price END AS Artefact_Price_nn
FROM 
    final_query
WHERE 
    Sell_Price IS NOT NULL
    AND Missing_Artefact = 0
    AND Missing_Second_Resource = 0
    AND Main_Resource_Price > 0
    AND Journal_Empty_Price IS NOT NULL
    AND Journal_Full_Price IS NOT NULL
    AND Sell_City = 'Black Market';
    
 """, Albion_conn)

query_check_2= time.time()
print(f'{query_check_2-query_check_1} seconds to complete')

BM_Craft

1.0151889324188232 seconds to complete


,index,Item_Placeholder,Tier,Enchant,Item_Name,Main_Resource,Main Resource Quantity,Secondary_Resource,Secondary_Resource_Quantity,Artefact_Placeholder,...,Secondary_Resource_Date,Artefact_Price,Artefact_Date,Journal_Empty_Price,Journal_Full_Price,Missing_Artefact,Missing_Second_Resource,Tier_Enchant,Secondary_Resource_Price_nn,Artefact_Price_nn
0,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-07-21T08:35:00,NaN,None,2585,2809,0,0,4.0,141,0
1,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-07-21T13:40:00,NaN,None,2585,2809,0,0,4.0,118,0
2,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-07-21T13:35:00,NaN,None,2585,2809,0,0,4.0,143,0
3,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-07-21T08:25:00,NaN,None,2585,2809,0,0,4.0,129,0
4,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-07-21T12:05:00,NaN,None,2585,2809,0,0,4.0,122,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65620,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,46836.0,2023-07-07T14:40:00,18756,112657,0,0,8.3,0,46836
65621,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,46836.0,2023-07-07T14:40:00,18756,112657,0,0,8.3,0,46836
65622,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,46836.0,2023-07-07T14:40:00,18756,112657,0,0,8.3,0,46836
65623,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,46836.0,2023-07-07T14:40:00,18756,112657,0,0,8.3,0,46836


In [14]:
# moving back to Pandas for final non-merge operations

BM_Craft['Secondary_Resource_Price'] = BM_Craft['Secondary_Resource_Price_nn']
BM_Craft = BM_Craft.drop('Secondary_Resource_Price_nn', axis=1)

BM_Craft['Artefact_Price'] = BM_Craft['Artefact_Price_nn']
BM_Craft = BM_Craft.drop('Artefact_Price_nn', axis=1)

BM_Craft['Resource_Costs'] = (BM_Craft['Main Resource Quantity']*BM_Craft['Main_Resource_Price'])+(BM_Craft['Secondary_Resource_Quantity']*BM_Craft['Secondary_Resource_Price'])
BM_Craft['Artefact_Costs'] = BM_Craft['Artefact_Quantity']*BM_Craft['Artefact_Price']
BM_Craft['Journal_Profit'] = ((BM_Craft['Journal_Full_Price']*0.975)-BM_Craft['Journal_Empty_Price'])*(BM_Craft['Crafting Fame']/BM_Craft['Journal_EXP_Fill'])

BM_Craft['Profit_City'] = (BM_Craft['Sell_Price']*0.945)-BM_Craft['Artefact_Costs']-(BM_Craft['Resource_Costs']*0.752)+BM_Craft['Journal_Profit']
BM_Craft['Profit_City_Focus'] = (BM_Craft['Sell_Price']*0.945)-BM_Craft['Artefact_Costs']-(BM_Craft['Resource_Costs']*0.502)+BM_Craft['Journal_Profit']
BM_Craft['Profit_BZ_Q3'] = (BM_Craft['Sell_Price']*0.945)-BM_Craft['Artefact_Costs']-(BM_Craft['Resource_Costs']*0.598)+BM_Craft['Journal_Profit']
BM_Craft['ROI_BZ_Q3'] = BM_Craft['Profit_BZ_Q3']/BM_Craft['Sell_Price']

BM_Craft = BM_Craft[['Item_Name','City_Bonus','Tier','Enchant','Tier_Enchant','Profit_City','Profit_City_Focus','Profit_BZ_Q3','ROI_BZ_Q3','Sell_City','Sell_Price','Sell_Price_Date','Main_Resource','Main_Resource_Price','Main_Resource_City','Main_Resource_Date','Secondary_Resource','Secondary_Resource_Price','Secondary_Resource_City','Secondary_Resource_Date','Artefact_Costs','Journal_Profit','Resource_Costs']]

In [15]:
BM_Craft.columns

Index(['Item_Name', 'City_Bonus', 'Tier', 'Enchant', 'Tier_Enchant',
       'Profit_City', 'Profit_City_Focus', 'Profit_BZ_Q3', 'ROI_BZ_Q3',
       'Sell_City', 'Sell_Price', 'Sell_Price_Date', 'Main_Resource',
       'Main_Resource_Price', 'Main_Resource_City', 'Main_Resource_Date',
       'Secondary_Resource', 'Secondary_Resource_Price',
       'Secondary_Resource_City', 'Secondary_Resource_Date', 'Artefact_Costs',
       'Journal_Profit', 'Resource_Costs'],
      dtype='object')

In [16]:
BM_Craft.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\crafting_report.csv", index=False)

In [17]:
Caerleon_Transport.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\transport_report.csv", index=False)